In [ ]:
# FEniCSx version of continuum learning simulation

import numpy as np
import matplotlib.pyplot as plt
import importlib

import funcs_proj_smooth, calc, bcs, plot_funcs


In [ ]:
# --- Parameters ---
num_cycles = 20
num_tasks = 2
save_to_table = False
iterations = 20
if_periodic = False
# update_type = 'Loss'
# update_type = 'BCs times Loss'
update_type = 'Loss integral'
# update_type = 'dumb inputs'
shared_loss = False
c_type = 'tensor'
# c_type = 'scalar'

x_min, x_max = -0.5, 0.5
y_min, y_max = -3.5, 3.5
Ngrid = 100

alpha = 1.0
beta = 0.0
gamma_Q = 0.1
gamma_c = -0.02
# gamma_c = -0.0
gamma_decay = -0.01
# gamma_decay = -0.0
# y1 = y2 = 1.0
# yout1 = yout2 = -1.0
y1_in = 1.0
y2_in = -1.0
y1_target = -1.0
y2_target = 1.0
sigma_in = 0.05
sigma_target = 0.1
offset_in = 1.0
offset_target = 1.5
c0 = 1.0
c_ss = 0.01

In [ ]:
# --- Initiate Mesh ---
import MeshClass
importlib.reload(MeshClass)
from MeshClass import MeshClass

Mesh = MeshClass(x_min, x_max, y_min, y_max, Ngrid)

In [ ]:
# --- Initiate Dolfinx Functionspace ---
import FuncspaceClass
importlib.reload(FuncspaceClass)
from FuncspaceClass import FuncspaceClass

Funcspace = FuncspaceClass(Mesh)

In [ ]:
# --- Build Mesh DOFs due to Functionspace ---
Mesh.build_dofs(Funcspace)

In [ ]:
# --- Initiate Supervisor variables ---
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass
importlib.reload(plot_funcs)

Supervisor = SupervisorClass()

# # --- Plot input and target ---

Supervisor.pose_input_val(y1_in, sigma_in, offset_in, Mesh, Funcspace)
Supervisor.pose_output_val(Mesh, Funcspace)
Supervisor.pose_target_val(y1_target, sigma_target, offset_target, Mesh, Funcspace)

plot_funcs.inputs_and_target(Supervisor, Mesh)

In [ ]:
# --- Initiate Material State variables ---
from ufl import dot, sqrt, outer, Identity, as_tensor
from dolfinx.fem import Function
import StateClass
importlib.reload(StateClass)
from StateClass import StateClass

c_ss = 0.01 * Identity(2)

State = StateClass(gamma_Q, gamma_c, gamma_decay, c_ss, c_type)

c = Function(Funcspace.ScalarFuncSpace)
c.interpolate(lambda x: np.full(x.shape[1], c0))
if c_type == 'tensor':
    # c = c * Identity(2)
    # c = as_tensor([[1.0*c, 0.1*c], [0.1*c, 1.0*c]])
    c = c * as_tensor([[0.05, 0.0], [0.0, 1.0]])


State.init_c(c0, c_type, Funcspace, upload_from_file=False, c=c)  # conductivity field c

In [ ]:
# --- Dynamics Loop ---
importlib.reload(bcs)
importlib.reload(calc)
importlib.reload(funcs_proj_smooth)
importlib.reload(plot_funcs)
from ufl import dot, sqrt, outer, Identity, as_tensor

Supervisor.init_loss()
# plot_funcs.c_tensor(State, Mesh, iteration=1, cycle=1)

for j in range(num_cycles):
    for i in range(iterations):
        #  --- Measurement modality  ---

        # task 1
        if j % num_tasks == 0:
            Supervisor.pose_input_val(y1_in, sigma_in, offset_in, Mesh, Funcspace)  # inputs
            Supervisor.pose_target_val(y1_target, sigma_target, offset_target, Mesh, Funcspace)  # target
            Supervisor.pose_output_val(Mesh, Funcspace)  # outputs

        # task 2
        elif j % num_tasks == 1:
            Supervisor.pose_input_val(y2_in, sigma_in, offset_in, Mesh, Funcspace)  # inputs 2nd task
            Supervisor.pose_output_val(Mesh, Funcspace)  # outputs 2nd task
            Supervisor.pose_target_val(y2_target, sigma_target, offset_target, Mesh, Funcspace)  # target 2nd task

        State.calc_Poisson(Supervisor, Mesh, Funcspace)  # p
        State.calc_Q(Funcspace, Mesh, smooth=True)  # Q

        # --- Loss ---
        Supervisor.calc_loss(State, Mesh, Funcspace)

        # ---- calculate BEASTAL update modality values ---
        Supervisor.calc_update(update_type, Mesh, Funcspace)

        # plot fields
        plot_funcs.measurement_fields(State, Supervisor, Mesh, iteration=i, cycle=j)
        # plot_funcs.Q(State, Mesh, update=False, iteration=1, cycle=1)

        #  --- Measurement modality 2  ---
        if shared_loss:
            # task 2
            if j % num_tasks == 1:
                Supervisor.pose_input_val(y1_in, sigma_in, offset_in, Mesh, Funcspace)
                Supervisor.pose_target_val(y1_target, sigma_target, offset_target, Mesh, Funcspace)
                Supervisor.pose_output_val(Mesh, Funcspace)

            # task 1
            elif j % num_tasks == 0:
                Supervisor.pose_input_val(y2_in, sigma_in, offset_in, Mesh, Funcspace)
                Supervisor.pose_output_val(Mesh, Funcspace)
                Supervisor.pose_target_val(y2_target, sigma_target, offset_target, Mesh, Funcspace)

            #  --- Measurement modality  ---
            State.calc_Poisson(Supervisor, Mesh, Funcspace)  # p
            State.calc_Q(Funcspace, Mesh, smooth=True)  # Q

            # --- Loss ---
            Supervisor.calc_loss(State, Mesh, Funcspace, num=2)

            # ---- calculate BEASTAL update modality values ---
            # this includes combining the updates between loss1 and loss2
            Supervisor.calc_update(update_type, Mesh, Funcspace, num=2)

            # plot fields - 2nd task, Measurement modality
            plot_funcs.measurement_fields(State, Supervisor, Mesh, iteration=i, cycle=j, num=2)
            # plot_funcs.Q(State, Mesh, update=False, iteration=1, cycle=1)

            Supervisor.combine_updates(Funcspace)
        Supervisor.append_loss(shared_loss)  # append to loss vector

        #  --- Update modality  ---
        State.calc_Poisson(Supervisor, Mesh, Funcspace, update=True)  # p
        State.calc_Q(Funcspace, Mesh, smooth=True, update=True)  # Q

        # evolve conductivity
        State.evolve_c(Funcspace)

        # plot fields - Update modality
        plot_funcs.update_fields(State, Supervisor, Mesh, iteration=i, cycle=j)
        plot_funcs.Q(State, Mesh, update=True, iteration=1, cycle=1)
        plot_funcs.c_tensor(State, Mesh, iteration=1, cycle=1)
        plot_funcs.Loss_vec(Supervisor)

In [ ]:
from ufl import TrialFunction, TestFunction
from ufl import dot, grad, div, dx
from dolfinx.fem import form
from dolfinx.fem import Function, dirichletbc, Constant
from dolfinx.fem import functionspace, locate_dofs_geometrical
importlib.reload(plot_funcs)

State.absQ_update

# plot_funcs.check_spline(Mesh, Supervisor.update.l_fn, array_comparison=Supervisor.update.l_array)
# plot_funcs.check_spline(Mesh, Supervisor.update.r_fn, array_comparison=Supervisor.update.r_array)

In [ ]:
np.save("c_latest.npy", State.c.x.array)